# Sieci rekurencyjne (RNN) - „zapamiętujące”

Rozpoznawanie mowy należy do zagadnień, w których kolejność występowania rozpoznawanych elementów - fonemów lub słów - ma istotne znaczenie dla zrozumienia analizowanej sekwencji. Każdy język można opisać zbiorem reguł, które określają, jak duże jest prawdopodobieństwo wystąpienia określonej sekwencji fonemów lub słów. Przykładowo w języku polskim (oraz innych językach słowiańskich) jest stosunkowo duże prawdopodobieństwo wystąpienia zbitek spółgłoskowych, podczas gdy w niektórych językach, np. japońskim, takie zbitki niemal nie występują.

Większość algorytmów uczenia maszynowego zakłada, że wejścia i wyjścia są niezależne od siebie, co w przypadku rozpoznawania nie zawsze jest założeniem zgodnym z prawdą. Algorytm służący do rozpoznawania mowy mógłby zawęzić obszar możliwych fonemów/słów, które mogą występować w kolejnych analizowanych fragmentach sygnału, gdyby posiadał pewnego rodzaju pamięć i zdolność do analizowania całej sekwencji danych, a nie jedynie aktualnie podawanego wektora cech. Przydatną cechą byłaby też możliwość analizowania sekwencji o różnej długości. Do algorytmów wykazujących takie cechy zaliczamy rekurencyjne sieci neuronowe (RNN, *recurrent neural network*). Znajdują one zastosowanie w dziedzinach, w których istotna jest kolejność występowania zjawisk, a więc właśnie w rozpoznawaniu mowy, analizie języka naturalnego czy analizie szeregów czasowych.

## Sieć rekurencyjna

Pojedynczy neuron sieci rekurencyjnej przyjmuje na wejściu nie tylko aktualnie analizowaną ramkę sygnału, ale też dane ze swojego wyjścia otrzymane w wyniku przejścia poprzedniej ramki. Dzięki temu na uzyskiwane wartości ma wpływ nie tylko aktualna ramka, lecz również ramki poprzednie - możemy więc mówić o pewnego rodzaju pamięci neuronu. Analogicznie wygląda sytuacja w przypadku złożenia kilku neuronów w warstwę - na wejście warstwy oprócz wektora wejściowego podawany jest wektor wyjściowy tej warstwy z poprzedniej chwili czasowej.

W sieci rekurencyjnej każdy neuron ma dwa zestawy wag - wagi dla wejść $W_x$ (odpowiadające ramce w chwili t) oraz wagi dla wyjść $W_y$ (odpowiadające ramce w chwili t-1). Wartości w wektorze wyjściowym w chwili t można wyznaczyć ze wzoru:

$y_{(t)} = \phi(X_{(t)}^{T}*W_x + y_{(t-1)}^{T}*W_y + b)$

gdzie $\phi$ jest funkcją aktywacji, $X_{(t)}$ jest wektorem wejściowym, $y_{(t-1)}$ jest wektorem wyjściowym, $b$ to bias warstwy, a $T$ oznacza transpozycję wektora.

Każdy neuron RNN ma jeszcze jeden element: stan ukryty (oznaczany $h_{(t)}$). Pełni on funkcję pamięci i zależy od wartości wejściowych oraz stanu ukrytego z poprzedniej chwili czasowej. W przypadku najprostszych sieci rekurencyjnych stan ukryty i wartość wyjściowa są sobie równe.

## Zanik pamięci

Wadą sieci zbudowanych z wielu prostych warstw rekurencyjnych jest omawiany już wcześniej problem zanikającego gradientu. W kontekście RNN można też mówić o „zaniku pamięci” - w przypadku długich sekwencji sieć RNN bierze wtedy pod uwagę jedynie kilka ostatnich ramek, a wpływ pierwszych ramek na stany ukryte komórek jest już praktycznie zerowy.

By rozwiązać problemu zaniku pamięci, zostały opracowane specjalne rodzaje komórek cechujących się pamięcia długotrwałą. Najbardziej znaną komórką tego typu jest LSTM (*long short-term memory*).

Stan ukryty komórki LSTM podzielony jest na dwa wektory:

- $h_{t}$ stanowiący stan krótkoterminowy,
- $c_{t}$ stanowiący stan długoterminowy.

Oba wektory stanu z chwili czasowej $t-1$ $(h_{t-1}$ i $c_{t-1})$ podawane są na wejście komórki. Oprócz tego LSTM przyjmuje wektor danych wejściowych $X_{t}$. Na wyjściu dostajemy z kolei wektor danych wyjściowych $y_{t} (=h_t)$ oraz stany $h_{t}$ i $c_{t}$.

![caption](https://www.mdpi.com/information/information-15-00517/article_deploy/html/images/information-15-00517-g002-550.jpg)

Wewnątrz komórki LSTM znajdują się dwa rodzaje warstw gęstych:

- warstwa główna, która analizuje bieżące wartości wejścia $x_{t}$ i poprzedniego (krótkoterminowego) stanu $h_{t-1}$. W prostej komórce rekurencyjnej wyście tej warstwy podawane jest od razu do wektorów $y_{t}$ i $h_{t}$. W komórkach LSTM jest natomiast przechowywane w wektorze $c_t$ - „kandydacie”, który następnie jest częściowo zapisywany w $c_{(t-1)}$ (stanie długoterminowym). Domyślną funkcją aktywacji warstwy głównej jest tangens hiperboliczny.
- trzy tzw. kontrolery bramkowe, które kontrolują działanie bramek. Ich funkcją aktywacji jest funkcja logistyczna ($\sigma$), a więc na ich wyjściu mogą pojawić się jedynie wartości z przedziału $(0,1)$. Jeżeli na wyjściu kontrolera otrzymamy 0, to sterowana przez niego bramka zostanie zamknięta i informacje zostaną usunięte, a jeżeli 1 -  informacje zostaną przekazane danej. Każdy kontroler kontroluje jedną bramkę - te bramki to:

    - bramka "zapomnij" (*forget gate*, $f_t$ na schemacie) - przez nią przechodzi wektor stanu $c_{t-)}$. Bramka decyduje, które "wspomnienia" (stany długoterminowe) należy zachować, a które zapomnieć;
    - bramka wejściowa (*input gate*, $i_t$ na schemacie) - kontroluje, które części $c_t$ powinny być dodane do stanu długoterminowego;
    - bramka wyjściowa (*output gate*, $o_t$ na schemacie) - kontroluje, które części stanu długoterminowego powinny być odczytywane i dodawane do $h_{t}$ i $y_{t}$.

Poszczególne wartości w komórce można obliczyć korzystając z następujących zależności:

$i_{t} = \sigma (W_{xi}^{T} \cdot x_{t} + W_{hi}^{T} \cdot h_{t-1} + b_i)$

$f_{t} = \sigma (W_{xf}^{T} \cdot x_{t} + W_{hf}^{T} \cdot h_{t-1} + b_f)$

$o_{t} = \sigma (W_{xo}^{T} \cdot x_{t} + W_{ho}^{T} \cdot h_{t-1} + b_o)$

$g_{t} = tanh (W_{xg}^{T} \cdot x_{t} + W_{hg}^{T} \cdot h_{t-1} + b_g)$

$c_{t} = f_{t} \odot c_{t-1} + i_{t} \odot g_{t}$

$y_{t} = h_{t} = o_{t} \odot tanh( c_{t})$

Symbol $\odot$ oznacza mnożenie element po elemencie (nie mnożenie macierzowe).

## GRU

Drugim często stosowanym rodzajem komórki rekurencyjnej jest GRU (*gated recurrent unit*). Komórki GRU są uproszczoną wersją komórek LSTM, jednak pomimo prostszej budowy często dają równie dobre rezultaty. W stosunku do komórek LSTM można określić cztery różnice:

- występuje tylko jeden wektor stanu $h_{t}$, nie ma podziału na stan krótko- i długoterminowy;
- bramka "zapomnij" i bramka wejściowa kontrolowane są przez jeden kontroler (*reset gate*, $r_t$) - jeżeli na wyjściu kontrolera jest 0, to bramka wejściowa jest otwarta, a bramka "zapomnij" zamknięta, jeżeli na wyjściu jest 1 - bramka wejściowa jest zamknięta, a bramka "zapomnij" otwarta;
- nie ma bramki wyjściowej - za każdym razem cały wektor stanu podawany na wyjście komórki;
- występuje dodatkowy kontroler bramkowy (*update gate*, $z_t$), który kontroluje, jaka część stanu poprzedniego będzie podawana do warstwy głównej.

![caption](https://www.mdpi.com/information/information-15-00517/article_deploy/html/images/information-15-00517-g005-550.jpg)

Wektor stanu komórki GRU można wyznaczyć wykorzystując następujące zależności:

$z_{t} = \sigma (W_{xz}^{T} \cdot x_{t} + W_{hz}^{T} \cdot h_{t-1} + b_z)$

$r_{t} = \sigma (W_{xr}^{T} \cdot x_{t} + W_{hr}^{T} \cdot h_{t-1} + b_r)$

$\tilde h_{t} = tanh (W_{xh}^{T} \cdot x_{t} +  r_{t} \odot (W_{hh}^{T} \cdot h_{t-1}) + b_h)$

$h_{t} = z_{t} \odot \tilde h_{t-1} + (1 - z_{t}) \odot h_{t-1}$

## Implementacja RNN w pytorchu

Zaczynamy od wczytania danych, na których pracowaliśmy już podczas omawiania sieci z warstwami gęstymi oraz konwolucyjnych.

In [1]:
!pip install pytorch-ignite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 5.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from ignite.engine import create_supervised_trainer, create_supervised_evaluator, Events
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers import ProgressBar
from ignite.handlers import FastaiLRFinder

/usr/local/lib/python3.11/dist-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
folder = 'drive/MyDrive/PUM/Lab8-9/' #zmodyfikuj ścieżkę odpowiednio do lokalizacji plików
feats = np.load(folder+'melspec_feats.npy')
labels = np.load(folder+'labels.npy')

In [ ]:
feats = feats.reshape(feats.shape[0], 1, feats.shape[1], feats.shape[2]) #zmiana kształtu macierzy na taki, z którego można bezproblemowo utworzyć jednokanałowy tensor
feats = feats.astype(np.float32)

feats.shape

(10500, 1, 99, 26)

Macierz ma na razie wymiary `liczba_sygnałów` $\times$ `liczba_kanałów_tensora` $\times$ `liczba_ramek` $\times$ `liczba_cech`. Jeżeli chcemy podać je do sieci rekurencyjnej, powinniśmy najpierw mieć liczbę cech, a potem liczbę ramek. Musimy więc transponować macierz:

In [6]:
feats = np.transpose(feats, (0,1,3,2))
feats.shape

(10500, 1, 26, 99)

Następnie dzielimy dane na zbiory, tworzymy datasety i loadery. Definiujemy też rozmiar wsadu (`batch_size`).

In [ ]:
batch_size = 50

X_train, X_val_test, y_train, y_val_test = train_test_split(feats, labels, random_state=42,
                                                            stratify=labels, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state=42,
                                                stratify=y_val_test, train_size=0.5)

trainset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
valset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
testset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

train_loader = DataLoader(trainset, batch_size=batch_size)
val_loader = DataLoader(valset, batch_size=batch_size)
test_loader = DataLoader(testset, batch_size=batch_size)

Teraz przechodzimy do definicji klasy, w której definiowana jest sieć. Poniżej przykład sieci konwolucyjno-rekurencyjnej z komórką typu GRU z odpowiednio dobranymi wartościami hiperparametrów - czyli tak, by kod się uruchomił, dobrymi wynikami zajmiemy się za chwilę.

In [ ]:
class NetGRU(nn.Module):
    def __init__(self):
        super(NetGRU, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=10, stride=1, padding=0)
        self.gru1 = nn.GRU(input_size=10, hidden_size=20, num_layers=1,
                           batch_first=True, dropout=0.5, bidirectional=False)
        self.fc1 = nn.Linear(20, 40)
        self.fc2 = nn.Linear(40, 35)
        self.drop1 = nn.Dropout(p=0.5)

    def forward(self, x):
#         print(x.shape)
        x = F.relu(F.max_pool2d(self.conv1(x), 4))
#         print(x.shape)
        x = x.view(batch_size,-1, 10) #zmiana kształtu danych - działa podobnie do funkcji np.reshape
        #musimy zmienić kształt danych, ponieważ na wyjściu warstwy konwolucyjnej mamy dane 4D (batch_size x liczba kanałów x liczba_cech x liczba_ramek) a warstwa rekurencyjna przyjmuje dane 3D (batch_size x długość_sekwencji x input_size)
#         print(x.shape)
        x, h = self.gru1(x)
#         print(x.shape)
        x = self.fc1(x[:,-1]) #warstwa gęsta przyjmuje dane 2D (batch_size x łączna liczba cech),a warstwa GRU zwraca dane 3D, więc musimy zrzutować dane do macierzy 2D
#         print(x.shape)
        x = self.drop1(x)
#         print(x.shape)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Hiperparametry, które trzeba zdefiniować w warstwie GRU to:

- `input_size` - musi być równy drugiemu wymiarowi danych wyjściowych z poprzedniej warstwy (pierwszym wymiarem jest rozmiar batcha). Tutaj poprzednią warstwą jest warstwa konwolucyjna, więc `input_size` jest równe liczbie kanałów wyjściowych warstwy `nn.Conv2d`.
- `hidden_size` - liczba cech w stanie ukrytym $h$, oznacza też rozmiar danych na wyjściu warstwy GRU (i przy okazji danych wejściowych kolejnej warstwy).

Pozostałe hiperparametry nie muszą być definiowane przez użytkownika, ponieważ posiadają wartości domyślne:

- `num_layers` - liczba warstw rekurencyjnych, domyślnie wynosi 1. Jeżeli $>1$, to jest to równoważne z utworzeniem kilku warstw GRU (tylu, ile wynosi `num_layers`),
- `batch_first` - określona postać (kolejność wymiarów) tensora wyjściowego; domyślnie `False`. Jeżeli `batch_first=False`, to na wyjściu otrzymujemy tensor o wymiarach (długość_sekwencji, batch_size, liczba_cech); jeżeli `batch_first=True`, to otrzymujemy tensor o wymiarach (batch_size, długość_sekwencji, liczba_cech),
- `dropout` - domyślnie wynosi 0,
- `bidirectional` - określa, czy przepływ informacji ma być jednokierunkowy (`False`, wartość domyślna), czy dwukierunkowy. Jeżeli `bidirectional=True`, to podczas analizy danej ramki brane są pod uwagę zarówno poprzednie, jak i kolejne ramki.

W analogiczny sposób definiuje się sieć z warstwą typu LSTM:

In [ ]:
class NetLSTM(nn.Module):
    def __init__(self):
        super(NetLSTM, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=10, stride=1, padding=0)
        self.lstm = nn.LSTM(input_size=10, hidden_size=20, num_layers=1,
                           batch_first=True, dropout=0.5, bidirectional=False)
        self.fc1 = nn.Linear(20, 40)
        self.fc2 = nn.Linear(40, 35)
        self.drop1 = nn.Dropout(p=0.5)

    def forward(self, x):
#         print(x.shape)
        x = F.relu(F.max_pool2d(self.conv1(x), 4))
#         print(x.shape)
        x = x.view(batch_size,-1, 10) #zmiana kształtu danych - działa podobnie do funkcji np.reshape
                            #musimy zmienić kształt danych, ponieważ na wyjściu warstwy konwolucyjnej mamy dane 4D (batch_size x liczba kanałów x liczba_cech x liczba_ramek)
                            #a warstwa rekurencyjna przyjmuje dane 3D (batch_size x długość_sekwencji x input_size)
#         print(x.shape)
        x, h = self.lstm(x)
#         print(x.shape)
        x = self.fc1(x[:,-1]) #warstwa gęsta przyjmuje dane 2D (batch_size x łączna liczba cech), a warstwa LSTM zwraca dane 3D, więc musimy zrzutować dane do macierzy 2D
#         print(x.shape)
        x = self.drop1(x)
#         print(x.shape)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Po zdefiniowaniu klasy Net możemy przejść do treningu sieci. Korzystamy z kodu z poprzednich zadań - definiujemy `optimizer`, optymalizujemy `learning_rate`, inicjalizujemy `trainer` i `evaluator` oraz rozpoczynamy trening i ewaluację sieci. Tym razem skorzystamy z często wykorzystywanego przy większych modelach optymalizatora [Adam](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) (tutaj [oryginalny artykuł](https://arxiv.org/pdf/1412.6980) dla zainteresowanych).

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# print(device)
criterion = nn.NLLLoss()
model = NetLSTM()
model.to(device)
# optimizer = optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=1e-8)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


In [11]:
init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

In [12]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, metrics={"acc": Accuracy(), "loss": Loss(nn.NLLLoss())}, device=device)
ProgressBar(persist=True).attach(trainer, output_transform=lambda x: {"batch loss": x})

/usr/local/lib/python3.11/dist-packages/ignite/handlers/tqdm_logger.py:127: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [14]:
lr_finder = FastaiLRFinder()
to_save={'model': model, 'optimizer': optimizer}
with lr_finder.attach(trainer, to_save, diverge_th=1.05, start_lr=1e-8, end_lr=1e-2) as trainer_with_lr_finder:
    #domyślnie start_lr jest taki, jak określony w optimizerze, a end_lr=10
    trainer_with_lr_finder.run(train_loader)

print("Suggested LR", lr_finder.lr_suggestion())

[1/168]   1%|           [00:00<?]

Suggested LR 1.1787686347935873e-06


In [15]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['acc'], metrics['loss']))

lr_finder.apply_suggested_lr(optimizer)
print('Training with suggested lr: ', optimizer.param_groups[0]['lr'])
#trainer.run(trainloader, max_epochs=1000)
trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
print(evaluator.state.metrics)

Training with suggested lr:  1.1787686347935873e-06


[1/168]   1%|           [00:00<?]

Validation Results - Epoch: 1  Avg accuracy: 0.03 Avg loss: 3.58


[1/168]   1%|           [00:00<?]

Validation Results - Epoch: 2  Avg accuracy: 0.03 Avg loss: 3.58


...

Validation Results - Epoch: 148  Avg accuracy: 0.03 Avg loss: 3.56


[1/168]   1%|           [00:00<?]

Validation Results - Epoch: 149  Avg accuracy: 0.03 Avg loss: 3.56


[1/168]   1%|           [00:00<?]

Validation Results - Epoch: 150  Avg accuracy: 0.03 Avg loss: 3.56
{'acc': 0.02857142857142857, 'loss': 3.557660900297619}


# Zadanie

Spróbuj dobrać architekturę sieci tak, by uzyskać lepsze wyniki niż bez użycia warstw rekurencyjnych. W tym celu możesz:

- dodać wartwy (konwolucyjne, rekurencyjne, gęste),
- zastosować normalizację wsadową,
- zmienić hiperparametry warstw,
- dodać lub usunąć dropout przed wybranymi warstwami,
- zmienić funkcje aktywacji warstw.